# Unfaithful Medium-sized DAG
## 120521

In [8]:
from causalsage import *

![](dag-w-masking.png)

In [9]:
%display plain

In [10]:
var('delta gamma beta')

(delta, gamma, beta)

# Compute Overall Covariance Matrix from DAG

$$W_{ij} = W_{i \gets j}$$

In [11]:
W = matrix([[0, 0, 0], # to 0
            [beta, 0, 0], # to 1
            [delta, gamma, 0]]) # to 2
    
W

[    0     0     0]
[ beta     0     0]
[delta gamma     0]

In [12]:
P = (identity_matrix(len(W.columns())) - W).inverse()
P

[                 1                  0                  0]
[              beta                  1                  0]
[beta*gamma + delta              gamma                  1]

In [13]:
Cov_total = P*P.T

[Cov_total[i, i] for i in range(3)]

[1, beta^2 + 1, (beta*gamma + delta)^2 + gamma^2 + 1]

# Compute Partial Covariance Matrix from DAG

Recall that

$$
\begin{aligned}  
	 \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{ZX} \\
     \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{XZ}^{T}
\end{aligned}
$$

So you need: the indices for X and the indices for Z

You can then extract the relevant submatrices and perform the computation.

## Example: Compute Cov(0, 1 | 2)

In [14]:
targets = [0, 1]
condset = [2]

In [15]:
SigmaX = extract_submatrix(targets, targets, Cov_total)

SigmaX

[         1       beta]
[      beta beta^2 + 1]

In [16]:
SigmaXZ = extract_submatrix(targets, condset, Cov_total)
SigmaXZ

[               beta*gamma + delta]
[(beta*gamma + delta)*beta + gamma]

In [17]:
SigmaZ = extract_submatrix(condset, condset, Cov_total)

SigmaZ

[(beta*gamma + delta)^2 + gamma^2 + 1]

In [18]:
SigmaXgZ = SigmaX  - SigmaXZ*SigmaZ.inverse()*SigmaXZ.T
SigmaXgZ

[                                     -(beta*gamma + delta)^2/((beta*gamma + delta)^2 + gamma^2 + 1) + 1 -((beta*gamma + delta)*beta + gamma)*(beta*gamma + delta)/((beta*gamma + delta)^2 + gamma^2 + 1) + beta]
[-((beta*gamma + delta)*beta + gamma)*(beta*gamma + delta)/((beta*gamma + delta)^2 + gamma^2 + 1) + beta               beta^2 - ((beta*gamma + delta)*beta + gamma)^2/((beta*gamma + delta)^2 + gamma^2 + 1) + 1]

$$
\begin{aligned}  
	 \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{ZX} \\
     \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{XZ}^{T}
\end{aligned}
$$

In [19]:
partial_covariance_matrix(targets, condset, Cov_total)

[                                     -(beta*gamma + delta)^2/((beta*gamma + delta)^2 + gamma^2 + 1) + 1 -((beta*gamma + delta)*beta + gamma)*(beta*gamma + delta)/((beta*gamma + delta)^2 + gamma^2 + 1) + beta]
[-((beta*gamma + delta)*beta + gamma)*(beta*gamma + delta)/((beta*gamma + delta)^2 + gamma^2 + 1) + beta               beta^2 - ((beta*gamma + delta)*beta + gamma)^2/((beta*gamma + delta)^2 + gamma^2 + 1) + 1]

# Compute Partial Regression Coefficients from DAG

$$\beta_{YX \cdot Z} = \frac{\gamma_{YX \cdot Z}}{\sigma_{X \cdot Z}^{2}}$$

In [25]:
print(partial_regression_coefficient(response=2, predictor=1, condset=[], Sigma=Cov_total).full_simplify())

print(partial_regression_coefficient(response=2, predictor=1, condset=[0], Sigma=Cov_total).full_simplify())

(beta*delta + (beta^2 + 1)*gamma)/(beta^2 + 1)
gamma


In [26]:
print(partial_regression_coefficient(response=2, predictor=0, condset=[], Sigma=Cov_total).full_simplify())

print(partial_regression_coefficient(response=2, predictor=0, condset=[1], Sigma=Cov_total).full_simplify())

beta*gamma + delta
delta
